# Burp Config

In [38]:
import os
from pathlib import Path
import ssl
import certifi
# import urllib3

class BurpConfig:

    def __init__(self, burp_cer, proxy="http://127.0.0.1:8080", config_path="config/burp"):
        self.config_path = Path(config_path).absolute()
        self.config_path.mkdir(parents=True, exist_ok=True)
        self.certificate = self.create_custom_ca_bundle(burp_cer)
        self.status = "off"
        self.proxy = proxy
        
        # restoring point snapshot
        self.original_environ = dict(os.environ)
        self.original_ssl_context_function = ssl._create_default_https_context
    

    def on(self):
        if self.status == "off":
            self.set_proxy_env(self.proxy)
            self.set_ssl_env()
            self.patch_ssl_context()
        self.status = "on"

    def off(self):
        if self.status == "on":
            self.restore_env()
            self.unpatch_ssl_context()
        self.status = "off"

    def set_proxy_env(self, proxy):
        os.environ["ALL_PROXY"] = proxy
        os.environ["all_proxy"] = proxy
        os.environ["HTTP_PROXY"] = proxy
        os.environ["http_proxy"] = proxy
        os.environ["HTTPS_PROXY"] = proxy
        os.environ["https_proxy"] = proxy

    # Set custom CA bundle for SSL verification
    def set_ssl_env(self):
        os.environ['SSL_CERT_FILE'] = self.certificate
        os.environ['REQUESTS_CA_BUNDLE'] = self.certificate
        os.environ['CURL_CA_BUNDLE'] = self.certificate

    def restore_env(self):
        os.environ.clear()
        os.environ.update(self.original_environ)
    
    def patch_ssl_context(self):
        # Create a custom SSL context
        custom_context = ssl.create_default_context(cafile=self.certificate)
        # Replace the default SSL context
        ssl._create_default_https_context = lambda: custom_context
    
    def unpatch_ssl_context(self):
        ssl._create_default_https_context = self.original_ssl_context_function


    def create_custom_ca_bundle(self, burp_cer):
        burp_pem = str((self.config_path/"burp_certificate.pem"))
        all_pem  = str((self.config_path/"custom_ca_bundle.pem"))
        
        # Convert Burp certificate to PEM format
        os.system(f"openssl x509 -inform der -in {burp_cer} -out {burp_pem}")
        
        # Combine Burp certificate with system CA certificates
        with open(all_pem, 'wb') as outfile:
            with open(burp_pem, 'rb') as infile:
                outfile.write(infile.read())
            with open(certifi.where(), 'rb') as infile:
                outfile.write(infile.read())
        # $ cat {burp_pem} /etc/ssl/certs/ca-certificates.crt > {all_pem}
        
        return all_pem


    def __enter__(self):
        self.on()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.off()

    # def patch_urllib3(self):
    #     urllib3.util.ssl_.DEFAULT_CERTS = self.all_pem

    # def unpatch_urllib3(self):
    #     urllib3.util.ssl_.DEFAULT_CERTS = certifi.where()

In [39]:
# with BurpConfig("path/to/burp.cer") as burp:
#     # do

# burp = BurpConfig("path/to/burp.cer")
# burp.on()
# do

# burp.off()
# 

In [40]:
burp = BurpConfig("/home/ali/Storage/Network/Tools/BurpSuiteCommunity/burp.cer")
burp.on()
# burp.off()
burp.status

'on'

# utils

In [19]:
from pprint import pprint

In [4]:
def show(obj):
    print(
        *[x for x in dir(obj) if not x.startswith("_")],
        sep="\n"
    )

In [ ]:
def extract_columns(db):
    return list(db[0]["properties"].keys())

In [ ]:
def extract_type(obj):
    obj_type = check_data_type(obj)
    
    if check_data_type(obj) == "list":
        return extract_columns(obj)
    if "type" in obj.keys():
        return obj["type"]

In [ ]:
def check_data_type(obj):
    if isinstance(obj, list):
        return "list"
    elif isinstance(obj, dict):
        return "dict"
    elif isinstance(obj, (int, float)):
        return "number"
    elif isinstance(obj, str):
        return "string"
    else:
        return "unknown"

In [ ]:
def extract_type(obj):
    obj_type = check_data_type(obj)
    match obj_type:
        case "list":
            return None
        case "dict":
            return None
        case "number":
            return None
        case "string":
            return None
        case _:
            return None
    # if check_data_type(obj) == "list":
    #     return extract_columns(obj)
    # if "type" in obj.keys():
    #     return obj["type"]

In [36]:
def http_error(status):
    match status:
        case 400:
            return "Bad request"
        case 404:
            return "Not found"
        case 418:
            return "I'm a teapot"

        # If an exact match is not confirmed, this last case will be used if provided
        case _:
            return "Something's wrong with the internet"
        


In [37]:
http_error(400)

'Bad request'

# Notion Type Handle

In [ ]:
import json
import pandas as pd

def notion_to_pandas(notion_results):
    with open('./result.json', 'w', encoding='utf8') as f:
        json.dump(notion_results, f, ensure_ascii=False)

    df = pd.read_json('./result.json')
    df.properties.to_json('./result.json', orient='records', force_ascii=False)
    df = pd.read_json('./result.json')

    os.remove('./result.json')
    return df

In [ ]:
class NotionTypeHandler:

    @staticmethod
    def title(series):
        return series.apply(lambda x: x["title"][0]['text']['content'] if x["title"] else "Untitled")
    @staticmethod
    def checkbox(series):
        return series.apply(lambda x: x["checkbox"])
    @staticmethod
    def number(series):
        return series.apply(lambda x: x["number"])
    def formula(series):
        TYPE = series[0]["formula"]['type']
        return series.apply(lambda x: x["formula"][TYPE])
    @staticmethod
    def status(series):
        return series.apply(lambda x: x["status"]['name'])
    @staticmethod
    def date(series):
        return series.apply(lambda x: x["date"]['start'][:10] if x["date"] else "")
    @staticmethod
    def select(series):
        return series.apply(lambda x: x["select"]["name"] if x["select"] else "")
    @staticmethod
    def multi_select(series):
        def join_multi_select(x):
            res = ""
            for item in x["multi_select"]:
                res += item["name"] + ", "
            return res[:-2]
        return series.apply(join_multi_select)
    @staticmethod
    def rich_text(series):
        def join_plain_text(x):
            res = ""
            for item in x["rich_text"]:
                res += item["plain_text"]
            return res
        return series.apply(join_plain_text)
    @staticmethod
    def people(series):
        def join_people(x):
            res = ""
            for item in x["people"]:
                res += item["id"][:8] + "..., "
            return res[:-2]
        return series.apply(join_people)

    @staticmethod
    def convert_series(series):
        TYPE = series[0]['type']
        match TYPE:
            case "title":
                series = NotionTypeHandler.title(series)
            case "checkbox":  # | "number":
                series = NotionTypeHandler.checkbox(series)
            case "number":
                series = NotionTypeHandler.number(series)
            case "formula":
                series = NotionTypeHandler.formula(series)
            case "date":
                series = NotionTypeHandler.date(series)
            case "status":
                series = NotionTypeHandler.status(series)
            case "select":
                series = NotionTypeHandler.select(series)        
            case "multi_select":
                series = NotionTypeHandler.multi_select(series)
            case "rich_text":
                series = NotionTypeHandler.rich_text(series)
            case "people":
                series = NotionTypeHandler.people(series)
            case _:
                return None
        return series

    @staticmethod
    def convert_dataframe(df, column_list=None):
        if not column_list:
            column_list = df.columns
        new_columns = []
        for column in column_list:
            series = df[column]
            new_series = NotionTypeHandler.convert_series(series)
            if isinstance(new_series, pd.Series):
                new_columns.append(new_series)
                    
        new_df = pd.concat(new_columns, axis=1)
        return new_df

In [ ]:
# df = notion_to_pandas(db)
# df = NotionTypeHandler.convert_dataframe(df)
# df.to_csv(path, index=False)
# df

# Notion API

In [224]:
from notion_client import Client

In [225]:
secret = "secret_6DlTuXjTyvD0b4Zb2xNHCdnKsQp4JSOD8Fc1D9FMU9F"
notion = Client(auth=secret)

In [ ]:
response = notion.databases.query(database_id="4698ca12d56f43b6b95ebb528b2a0c1d")

In [ ]:
pprint(response)

In [8]:
show(notion)

blocks
client
close
comments
databases
logger
options
pages
request
search
users


In [9]:
# help(notion.search)

In [17]:
# notion.search()

In [18]:
# notion.search(query="Records")

In [16]:
show(notion.databases)

create
list
parent
query
retrieve
update


In [89]:
database_id = {
    "Projects": "9a18b46bef9e4a18ad52a399021b0734",
    "Records": "b138b3e32eb04580a1e8b0ea4f2b817d",
    "Vocabularies": "57cd397ea56040cd83da5c877000ffb5",
    "test": "4698ca12d56f43b6b95ebb528b2a0c1d",
}

In [44]:
databases = {}
for title, ID in database_id.items():
    databases[title] = notion.databases.query(database_id=ID)["results"]

In [ ]:
len(databases["Projects"])  # rows

3

In [45]:
pprint(databases["Projects"])

[{'archived': False,
  'cover': None,
  'created_by': {'id': 'ab471901-fb0c-4c01-9ce9-f6d362cc5563',
                 'object': 'user'},
  'created_time': '2024-07-06T19:25:00.000Z',
  'icon': None,
  'id': 'c2994820-ab0d-4016-b8b0-78de4675d97b',
  'in_trash': False,
  'last_edited_by': {'id': 'ab471901-fb0c-4c01-9ce9-f6d362cc5563',
                     'object': 'user'},
  'last_edited_time': '2024-07-06T19:25:00.000Z',
  'object': 'page',
  'parent': {'database_id': '9a18b46b-ef9e-4a18-ad52-a399021b0734',
             'type': 'database_id'},
  'properties': {'Done': {'id': 'Yop%40',
                          'rollup': {'function': 'checked',
                                     'number': None,
                                     'type': 'number'},
                          'type': 'rollup'},
                 'Name': {'id': 'title', 'title': [], 'type': 'title'},
                 'Number of Words': {'id': 'uH%7D%5D',
                                     'number': None,
              

In [185]:
databases["test"] = notion.databases.query(database_id="4698ca12d56f43b6b95ebb528b2a0c1d")["results"]

In [ ]:
import os
import json
import pandas as pd

from notion_client import Client

In [186]:
import json
with open('./result.json', 'w', encoding='utf8') as f:
    json.dump(databases["test"], f, ensure_ascii=False)

In [187]:
import pandas as pd

db = pd.read_json('./result.json')
db.properties.to_json('./result.json', orient='records', force_ascii=False)
db = pd.read_json('./result.json')

In [188]:
db

,Status,Formula,Tags,Number,Text,Person,Date,Select,Checkbox,Name
0,"{'id': 'EQph', 'type': 'status', 'status': {'i...","{'id': 'Qc%60%3D', 'type': 'formula', 'formula...","{'id': 'WXj%5E', 'type': 'multi_select', 'mult...","{'id': '%5D%3FiM', 'type': 'number', 'number': 1}","{'id': 'iHtT', 'type': 'rich_text', 'rich_text...","{'id': 'lDiI', 'type': 'people', 'people': [{'...","{'id': 'q%7B%60h', 'type': 'date', 'date': {'s...","{'id': 'suTF', 'type': 'select', 'select': {'i...","{'id': 'uThB', 'type': 'checkbox', 'checkbox':...","{'id': 'title', 'type': 'title', 'title': [{'t..."
1,"{'id': 'EQph', 'type': 'status', 'status': {'i...","{'id': 'Qc%60%3D', 'type': 'formula', 'formula...","{'id': 'WXj%5E', 'type': 'multi_select', 'mult...","{'id': '%5D%3FiM', 'type': 'number', 'number':...","{'id': 'iHtT', 'type': 'rich_text', 'rich_text...","{'id': 'lDiI', 'type': 'people', 'people': []}","{'id': 'q%7B%60h', 'type': 'date', 'date': None}","{'id': 'suTF', 'type': 'select', 'select': None}","{'id': 'uThB', 'type': 'checkbox', 'checkbox':...","{'id': 'title', 'type': 'title', 'title': []}"
2,"{'id': 'EQph', 'type': 'status', 'status': {'i...","{'id': 'Qc%60%3D', 'type': 'formula', 'formula...","{'id': 'WXj%5E', 'type': 'multi_select', 'mult...","{'id': '%5D%3FiM', 'type': 'number', 'number': 2}","{'id': 'iHtT', 'type': 'rich_text', 'rich_text...","{'id': 'lDiI', 'type': 'people', 'people': []}","{'id': 'q%7B%60h', 'type': 'date', 'date': {'s...","{'id': 'suTF', 'type': 'select', 'select': {'i...","{'id': 'uThB', 'type': 'checkbox', 'checkbox':...","{'id': 'title', 'type': 'title', 'title': []}"


In [205]:
db["Person"] # series of dict
db["Person"][0] # dict
db["Person"][0]["people"]

[{'object': 'user', 'id': 'ab471901-fb0c-4c01-9ce9-f6d362cc5563'}]

In [222]:
NotionTypeHandler.convert_dataframe(db)

,Status,Formula,Tags,Number,Text,Person,Date,Select,Checkbox,Name
0,In progress,2,"mamad, kokab, naser",1.0,Rich Text,ab471901...,2024-07-15,mamad,False,mamad
1,Not started,0,,NaN,,,,,False,Untitled
2,Done,4,,2.0,,,2024-07-01,naser,True,Untitled


In [58]:
extract_columns(databases["Projects"])

['Upcoming',
 'Progress',
 'Sustainability',
 'Done',
 'Tags',
 'Record',
 'Vocabulary',
 'Number of Words',
 'Name']

In [59]:
extract_columns(databases["Records"])

['Created time', 'Tags', 'Number', 'Name']

In [74]:
book_1100_words = notion.databases.query(
    **{
        "database_id": database_id["Projects"],
        "filter": {
            "property": "Name",
            "title": {
                "contains": "1100"
            }
    }
})["results"][0]["properties"]

In [75]:
pprint(book_1100_words)

{'Done': {'id': 'Yop%40',
          'rollup': {'function': 'checked', 'number': 0, 'type': 'number'},
          'type': 'rollup'},
 'Name': {'id': 'title',
          'title': [{'annotations': {'bold': False,
                                     'code': False,
                                     'color': 'default',
                                     'italic': False,
                                     'strikethrough': False,
                                     'underline': False},
                     'href': None,
                     'plain_text': '1100 Words',
                     'text': {'content': '1100 Words', 'link': None},
                     'type': 'text'}],
          'type': 'title'},
 'Number of Words': {'id': 'uH%7D%5D', 'number': 1100, 'type': 'number'},
 'Progress': {'id': 'Mtr_',
              'rollup': {'function': 'checked', 'number': 0, 'type': 'number'},
              'type': 'rollup'},
 'Record': {'button': {}, 'id': '%5BqHq', 'type': 'button'},
 'Sustainabil

In [73]:
book_1100_words["properties"]["Sustainability"]["formula"]["number"]

0

In [151]:
def is_list(obj):
    return str(type(obj)) == str(type(list()))

def is_dict(obj):
    return str(type(obj)) == str(type(dict()))

In [165]:
def extract_value(obj):
    ctype = extract_type(obj)
    sub_obj = obj[ctype]
    if sub_obj:
        if is_list(sub_obj):
            inner_obj = sub_obj[0]
        elif is_dict(sub_obj):
            inner_obj = sub_obj
        else:
            return sub_obj
        dtype = extract_type(inner_obj)
        return inner_obj[dtype]
    return None

In [166]:
book_1100_words["Vocabulary"]
# extract_type(book_1100_words["Name"])  # title
# str((book_1100_words["Name"]["title"])) == str(list())#[0]
# extract_type(book_1100_words["Name"]["title"][0])

{'id': '%60coz', 'type': 'relation', 'relation': [], 'has_more': False}

In [167]:
str({}) == str(dict())

True

In [168]:
row = {}
for column in extract_columns(databases["Projects"]):
    row[column] = extract_value(book_1100_words[column])

pprint(row)

{'Done': 0,
 'Name': {'content': '1100 Words', 'link': None},
 'Number of Words': 1100,
 'Progress': 0,
 'Record': None,
 'Sustainability': 0,
 'Tags': None,
 'Upcoming': 0,
 'Vocabulary': None}


In [180]:
def construct_row(page, columns): # obj = databases["Projects"]
    row = {}
    for column in columns:
        row[column] = extract_value(page[column])
    return row

In [179]:
db[0]["properties"]

{'Upcoming': {'id': 'DhP%40',
  'type': 'rollup',
  'rollup': {'type': 'number', 'number': None, 'function': 'checked'}},
 'Progress': {'id': 'Mtr_',
  'type': 'rollup',
  'rollup': {'type': 'number', 'number': None, 'function': 'checked'}},
 'Sustainability': {'id': 'W%3A%3Di',
  'type': 'formula',
  'formula': {'type': 'number', 'number': 0}},
 'Done': {'id': 'Yop%40',
  'type': 'rollup',
  'rollup': {'type': 'number', 'number': None, 'function': 'checked'}},
 'Tags': {'id': 'ZhtY', 'type': 'multi_select', 'multi_select': []},
 'Record': {'id': '%5BqHq', 'type': 'button', 'button': {}},
 'Vocabulary': {'id': '%60coz',
  'type': 'relation',
  'relation': [],
  'has_more': False},
 'Number of Words': {'id': 'uH%7D%5D', 'type': 'number', 'number': None},
 'Name': {'id': 'title', 'type': 'title', 'title': []}}

In [183]:
construct_row(db[2]["properties"], extract_columns(db))

{'Upcoming': None,
 'Progress': None,
 'Sustainability': 0,
 'Done': None,
 'Tags': None,
 'Record': None,
 'Vocabulary': None,
 'Number of Words': None,
 'Name': None}

In [185]:
response = notion.databases.query(database_id=database_id["Projects"])
db = response["results"]

table = []
for page in db:
    row = construct_row(page["properties"], extract_columns(db))
    table.append(row)
pprint(table)

[{'Done': None,
  'Name': None,
  'Number of Words': None,
  'Progress': None,
  'Record': None,
  'Sustainability': 0,
  'Tags': None,
  'Upcoming': None,
  'Vocabulary': None},
 {'Done': 0,
  'Name': {'content': '1100 Words', 'link': None},
  'Number of Words': 1100,
  'Progress': 0,
  'Record': None,
  'Sustainability': 0,
  'Tags': None,
  'Upcoming': 0,
  'Vocabulary': None},
 {'Done': None,
  'Name': None,
  'Number of Words': None,
  'Progress': None,
  'Record': None,
  'Sustainability': 0,
  'Tags': None,
  'Upcoming': None,
  'Vocabulary': None}]


In [186]:
import pandas as pd

In [187]:
pd.DataFrame(table)

,Upcoming,Progress,Sustainability,Done,Tags,Record,Vocabulary,Number of Words,Name
0,NaN,NaN,0,NaN,None,None,None,NaN,None
1,0.0,0.0,0,0.0,None,None,None,1100.0,"{'content': '1100 Words', 'link': None}"
2,NaN,NaN,0,NaN,None,None,None,NaN,None


In [8]:
test = notion.databases.retrieve("9a18b46bef9e4a18ad52a399021b0734")
# type(test) # dict

In [12]:
list(test.keys())

['object',
 'id',
 'cover',
 'icon',
 'created_time',
 'created_by',
 'last_edited_by',
 'last_edited_time',
 'title',
 'description',
 'is_inline',
 'properties',
 'parent',
 'url',
 'public_url',
 'archived',
 'in_trash',
 'request_id']

In [20]:
# test["title"]#[0]["plain_text"]
pprint(test["properties"])

{'Done': {'id': 'Yop%40',
          'name': 'Done',
          'rollup': {'function': 'checked',
                     'relation_property_id': '`coz',
                     'relation_property_name': 'Vocabulary',
                     'rollup_property_id': 'ZTfY',
                     'rollup_property_name': 'notify (Done)'},
          'type': 'rollup'},
 'Name': {'id': 'title', 'name': 'Name', 'title': {}, 'type': 'title'},
 'Number of Words': {'id': 'uH%7D%5D',
                     'name': 'Number of Words',
                     'number': {'format': 'number'},
                     'type': 'number'},
 'Progress': {'id': 'Mtr_',
              'name': 'Progress',
              'rollup': {'function': 'checked',
                         'relation_property_id': '`coz',
                         'relation_property_name': 'Vocabulary',
                         'rollup_property_id': 'Mfwk',
                         'rollup_property_name': 'Review (8)'},
              'type': 'rollup'},
 'Record': 

# pandas

In [31]:
import os
from notion_client import Client
import pandas as pd 
import json 

#definition object notion_pandas
class NotionPandas:
    def __init__(self, authh, database_idd):  
        try:
            self.notion =  Client(auth=authh)
            self.my_page = self.notion.databases.query(database_id=database_idd)
            
        except:
            print('Please check your auth and database_id')

    def change_user(self, auth):
            try:
                self.notion = Client(auth)
            
            except:
                print('Please check your auth')
    
    def change_database(self, database_id):
        try:
            self.my_page = self.notion.databases.query(database_id)
        except:
            print('Please check your database_id')
    
    def return_pandas(self, print_json = False):
        data = self.my_page['results']
        #save my_page to json file
        with open('./result.json', 'w', encoding='utf8') as f:
            json.dump(data, f, ensure_ascii=False)

        # Show page json structure
        if print_json:
          self.show_json()

        df = pd.read_json('./result.json')  

        df.properties.to_json('./result.json', orient='records', force_ascii=False)
        df = pd.read_json('./result.json')
        col = df.columns
        
        for c in col:
            tYpe = dict(df[c][0])['type']
            if tYpe == 'checkbox' or tYpe == 'number':
                df[c] = df[c].apply(lambda x: dict(x)[tYpe])
            
            elif tYpe == 'date':
                df[c] = df[c].apply(lambda x: dict(dict(x)[tYpe])['start'])
            
            elif tYpe == 'status':
                df[c] = df[c].apply(lambda x: dict(dict(x)[tYpe])['name'])
            
            elif tYpe == 'title':
                df[c] = df[c].apply(lambda x: dict(list(dict(x)[tYpe])[0])['text']['content'])
            
            elif tYpe == 'multi_select':
                col_dict = []
                for i in range(len(df[c])):
                  multi = df[c][i][tYpe]
                  multi_dict = []
                  for j in range(len(multi)):
                    multi_dict.append(multi[j]['name'])
                  col_dict.append(multi_dict)
                df[c]=col_dict
                df[c] = df[c].agg(', '.join) #remove list square brackets
            
            elif tYpe == 'rich_text':
                col_dict = []
                for i in range(len(df[c])):
                  multi = df[c][i][tYpe]
                  multi_dict = []
                  for j in range(len(multi)):
                    multi_dict.append(multi[j]['text']['content'])
                  col_dict.append(multi_dict)
                df[c]=col_dict
                df[c] = df[c].agg(', '.join) #remove list square brackets
        
        #delete ./result.json
        os.remove('./result.json')

        return df
    
    def to_csv(self, path):
        df = self.return_pandas()
        df.to_csv(path, index=False)
    
    def show_json(self):
        with open('./result.json', 'r') as json_file:
            json_object = json.load(json_file)
        print(json.dumps(json_object, indent=4))
                
                

In [32]:
secret = "secret_6DlTuXjTyvD0b4Zb2xNHCdnKsQp4JSOD8Fc1D9FMU9F"
database_id = {
    "Projects": "9a18b46bef9e4a18ad52a399021b0734",
    "Records": "b138b3e32eb04580a1e8b0ea4f2b817d",
    "Vocabularies": "57cd397ea56040cd83da5c877000ffb5",
}

In [33]:
np = NotionPandas(secret, database_id["Projects"])

In [34]:
df = np.return_pandas()

/tmp/ipykernel_5608/582255850.py:68: FutureWarning: using <built-in method join of str object at 0x74f24af9b630> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df[c] = df[c].agg(', '.join) #remove list square brackets


IndexError: list index out of range

In [ ]:
with open('./result.json', 'w', encoding='utf8') as f:
    json.dump(, f, ensure_ascii=False)

In [35]:
db

NameError: name 'db' is not defined